In [ ]:
import os 

In [ ]:
os.listdir('../input')

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../input/train.csv', parse_dates=[0])

In [ ]:
df.head()

In [ ]:
test = pd.read_csv('../input/test.csv', parse_dates=[0])

In [ ]:
test.head()

In [ ]:
df_all = df.append(test, sort=False)

In [ ]:
df_all['hour'] = df['datetime'].dt.hour

In [ ]:
import numpy as np

In [ ]:
df_all['count'] = np.log(df_all['count'] + 1)
df_all['registered'] = np.log(df_all['registered'] + 1)
df_all['casual'] = np.log(df_all['casual'] + 1)

In [ ]:
df_all.shape, df.shape, test.shape

In [ ]:
df_all.shape

In [ ]:
from fastai.imports import *
from fastai.structured import *

In [ ]:
add_datepart(df_all, 'datetime', drop=False)

In [ ]:
df_all.info()

In [ ]:
df = df_all[~df_all['count'].isnull()]
test = df_all[df_all['count'].isnull()]

In [ ]:
df.shape, test.shape

In [ ]:
train = df[df['datetimeDay'] <= 15]

In [ ]:
valid = df[df['datetimeDay'] > 15]

In [ ]:
train.shape, valid.shape

In [ ]:
feats = [c for c in df.columns if c not in ['casual', 'registered', 'count']]

In [ ]:
feats

In [ ]:
feats = ['season',
 'holiday',
 'workingday',
 'weather',
 'temp',
 'atemp',
 'humidity',
 'windspeed',
 'datetimeDayofweek',
        'hour',
        'datetimeYear']

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(n_estimators=100)

In [ ]:
rf.fit(train[feats], train['count'])

In [ ]:
rf.predict(valid[feats])

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(valid['count'], rf.predict(valid[feats])) ** (1/2)

In [ ]:
pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh()

In [ ]:
test['count'] = (np.exp(rf.predict(test[feats])) - 1)

In [ ]:
test[['datetime', 'count']].to_csv('submission.csv', index=False)